In [1]:
import os
import glob
import uuid
import boto3
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from operator import add

load_dotenv()

REGION = os.getenv('AWS_REGION')
MODEL_ID = os.getenv('BEDROCK_MODEL_ID')
ALIAS_ID = os.getenv('BEDROCK_AGENT_ALIAS_ID')

AGENT_IDS = {
    'MOGAM-Chem-Agent': os.getenv('BEDROCK_AGENT_CHEM'),
    'MOGAM-AI-Agent': os.getenv('BEDROCK_AGENT_AI'),
    'MOGAM-LNP-Agent': os.getenv('BEDROCK_AGENT_LNP'),
    'MOGAM-BI-Agent': os.getenv('BEDROCK_AGENT_BI'),
    'MOGAM-Lead-Agent': os.getenv('BEDROCK_AGENT_LEAD'),
}

print(f'Region: {REGION}')
print(f'Model: {MODEL_ID}')
print(f'Agents: {list(AGENT_IDS.keys())}')

Region: us-west-2
Model: us.anthropic.claude-sonnet-4-5-20250929-v1:0
Agents: ['MOGAM-Chem-Agent', 'MOGAM-AI-Agent', 'MOGAM-LNP-Agent', 'MOGAM-BI-Agent', 'MOGAM-Lead-Agent']


In [2]:
# Configuration: True = ChatBedrock (LLM direct), False = Bedrock Agents (managed)
USE_LLM_DIRECT = True

# Load available papers
papers = glob.glob('../data/papers/*.pdf')
print(f'Found {len(papers)} papers')

paper_context = """
Available research papers:
1. GenMol: A Drug Discovery Generalist with Discrete Diffusion (2025)
2. A Deep Generative Model for the Design of Synthesizable Ionizable Lipids (2024)
3. Generative AI for designing and validating easily synthesizable and structurally novel antibiotics (2024)
4. NeurIPS: Generative model for synthesizing ionizable lipids - MCTS approach (2024)
5. Accelerating ionizable lipid discovery for mRNA delivery using machine learning and combinatorial chemistry (2024)

Focus: AI-driven ionizable lipid design using Monte Carlo Search Tree and Discrete Diffusion approaches.
"""

Found 0 papers


In [3]:
# State definition
class MeetingState(TypedDict):
    question: str
    paper_context: str
    chem_input: str
    ai_input: str
    lnp_input: str
    bi_input: str
    synthesis: str

In [4]:
# Agent helpers
bedrock_runtime = boto3.client('bedrock-agent-runtime', region_name=REGION)
llm = ChatBedrock(model_id=MODEL_ID, region_name=REGION)

def invoke_bedrock_agent(agent_id, prompt):
    """Call a managed Bedrock Agent."""
    response = bedrock_runtime.invoke_agent(
        agentId=agent_id,
        agentAliasId=ALIAS_ID,
        sessionId=str(uuid.uuid4()),
        inputText=prompt,
    )
    result = ''
    for event in response['completion']:
        if 'chunk' in event and 'bytes' in event['chunk']:
            result += event['chunk']['bytes'].decode('utf-8')
    return result

def ask_expert(role, agent_key, prompt, state):
    """Route to LLM direct or managed agent based on config."""
    full_prompt = f"{state['paper_context']}\n\nFrom a {role} perspective: {prompt}"
    if USE_LLM_DIRECT:
        return llm.invoke(full_prompt).content
    else:
        return invoke_bedrock_agent(AGENT_IDS[agent_key], full_prompt)

print('✓ Agent helpers ready')

✓ Agent helpers ready


In [5]:
# LangGraph node functions
def chemical_expert_node(state: MeetingState) -> dict:
    print('🧪 Chemical Expert thinking...')
    result = ask_expert('chemical structure', 'MOGAM-Chem-Agent', state['question'], state)
    print('✓ Chemical Expert done')
    return {'chem_input': result}

def ai_expert_node(state: MeetingState) -> dict:
    print('🤖 AI Expert thinking...')
    result = ask_expert('AI/ML', 'MOGAM-AI-Agent', state['question'], state)
    print('✓ AI Expert done')
    return {'ai_input': result}

def lnp_expert_node(state: MeetingState) -> dict:
    print('💊 LNP Expert thinking...')
    result = ask_expert('LNP formulation', 'MOGAM-LNP-Agent', state['question'], state)
    print('✓ LNP Expert done')
    return {'lnp_input': result}

def bi_expert_node(state: MeetingState) -> dict:
    print('🧬 Bioinformatics Expert thinking...')
    result = ask_expert('bioinformatics', 'MOGAM-BI-Agent', state['question'], state)
    print('✓ Bioinformatics Expert done')
    return {'bi_input': result}

def lead_synthesis_node(state: MeetingState) -> dict:
    print('👔 Team Lead synthesizing...')
    synthesis_prompt = f"""Based on the following expert inputs, provide a comprehensive project plan:

Chemical Expert: {state['chem_input']}
AI Expert: {state['ai_input']}
LNP Expert: {state['lnp_input']}
Bioinformatics Expert: {state['bi_input']}

Synthesize these perspectives and provide:
1. Key priorities
2. Implementation roadmap
3. Resource requirements
4. Risk mitigation strategies"""

    if USE_LLM_DIRECT:
        result = llm.invoke(synthesis_prompt).content
    else:
        result = invoke_bedrock_agent(AGENT_IDS['MOGAM-Lead-Agent'], synthesis_prompt)
    print('✓ Synthesis done')
    return {'synthesis': result}

print('✓ Node functions defined')

✓ Node functions defined


In [6]:
# Build the LangGraph
workflow = StateGraph(MeetingState)

# Add nodes
workflow.add_node('chemical_expert', chemical_expert_node)
workflow.add_node('ai_expert', ai_expert_node)
workflow.add_node('lnp_expert', lnp_expert_node)
workflow.add_node('bi_expert', bi_expert_node)
workflow.add_node('lead_synthesis', lead_synthesis_node)

# Experts run in parallel from START, then converge to lead
workflow.add_edge(START, 'chemical_expert')
workflow.add_edge(START, 'ai_expert')
workflow.add_edge(START, 'lnp_expert')
workflow.add_edge(START, 'bi_expert')
workflow.add_edge('chemical_expert', 'lead_synthesis')
workflow.add_edge('ai_expert', 'lead_synthesis')
workflow.add_edge('lnp_expert', 'lead_synthesis')
workflow.add_edge('bi_expert', 'lead_synthesis')
workflow.add_edge('lead_synthesis', END)

graph = workflow.compile()
print('✓ LangGraph compiled')

✓ LangGraph compiled


In [7]:
# Run the meeting
research_question = "We need to design efficient and novel ionizable lipid using AI. What are the key considerations?"

result = graph.invoke({
    'question': research_question,
    'paper_context': paper_context,
    'chem_input': '',
    'ai_input': '',
    'lnp_input': '',
    'bi_input': '',
    'synthesis': '',
})

print('\n' + '=' * 60)
print('MEETING COMPLETE')
print('=' * 60)

🤖 AI Expert thinking...
🧬 Bioinformatics Expert thinking...
🧪 Chemical Expert thinking...
💊 LNP Expert thinking...
✓ LNP Expert done
✓ Bioinformatics Expert done
✓ AI Expert done
✓ Chemical Expert done
👔 Team Lead synthesizing...
✓ Synthesis done

MEETING COMPLETE


In [ ]:
# Display expert outputs
for label, key in [('🧪 Chemical Expert', 'chem_input'), ('🤖 AI Expert', 'ai_input'),
                    ('💊 LNP Expert', 'lnp_input'), ('🧬 Bioinformatics Expert', 'bi_input')]:
    print(f'\n{label} ({len(result[key])} chars)')
    print('-' * 40)
    print(result[key][:500] + '...' if len(result[key]) > 500 else result[key])


🧪 Chemical Expert (3884 chars)
----------------------------------------
# Key Considerations for AI-Driven Ionizable Lipid Design

## 1. **Structural Components & Chemical Architecture**

### Critical Functional Elements:
- **Ionizable head group**: Must protonate at endosomal pH (~5-6) but remain neutral at physiological pH (~7.4)
  - Typical pKa range: 6.0-7.0
  - Common motifs: tertiary amines, piperazines, imidazoles
  
- **Linker region**: Connects head to tails; influences biodegradability
  - Ester bonds for enzymatic cleavage
  - Balance between stability an...

🤖 AI Expert (4196 chars)
----------------------------------------
# Key AI/ML Considerations for Ionizable Lipid Design

## 1. **Representation & Chemical Space**

### Molecular Encoding
- **SMILES vs. Graph-based representations**: Choose between string-based (SMILES) or graph neural networks (GNNs)
- **Fragment-based approaches**: Decompose lipids into functional components (head groups, linkers, tails)
- **Hierarchi

In [9]:
# Team Lead Synthesis
print('👔 TEAM LEAD SYNTHESIS')
print('=' * 60)
print(result['synthesis'])

👔 TEAM LEAD SYNTHESIS
# Comprehensive Project Plan: AI-Driven Ionizable Lipid Design for LNP-Based mRNA Delivery

## Executive Summary

This project aims to develop an AI-driven platform for discovering novel ionizable lipids optimized for mRNA delivery via lipid nanoparticles (LNPs). By integrating discrete diffusion models and Monte Carlo Tree Search (MCTS) with domain-specific constraints, we will accelerate the discovery of chemically valid, synthesizable, and high-performing lipid candidates that balance efficacy, safety, and manufacturability.

---

## 1. KEY PRIORITIES

### 1.1 Primary Objectives (Must-Have)
1. **Transfection Efficiency**: Generate lipids achieving ≥80% of benchmark performance (e.g., ALC-0315, SM-102)
2. **Safety Profile**: Cytotoxicity CC50 >100 μM; minimal immunogenicity
3. **Synthesizability**: ≤5 synthetic steps from commercial building blocks
4. **Chemical Validity**: pKa 6.0-7.0, MW 400-800 Da, biodegradable linkers

### 1.2 Secondary Objectives (Should-H

## Summary

This notebook uses **LangGraph** to orchestrate a multi-agent research meeting:

- **4 expert agents** run in parallel (Chemical, AI/ML, LNP, Bioinformatics)
- **1 lead agent** synthesizes all expert inputs
- Supports both **ChatBedrock (LLM direct)** and **Bedrock Agents (managed)** modes
- Configuration loaded from `.env` file